# Lesson 1 - Semantic Search

Welcome to Lesson 1. 

To access the `requirement.txt` file, go to `File` and click on `Open`.
 
I hope you enjoy this course!

### Import the Needed Packages

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
from DLAIUtils import Utils
import DLAIUtils

import os
import time
import torch

In [3]:
from tqdm.auto import tqdm

### Load the Dataset

In [4]:
dataset = load_dataset('quora', split='train[240000:290000]')

In [5]:
dataset[:5]

{'questions': [{'id': [207550, 351729],
   'text': ['What is the truth of life?', "What's the evil truth of life?"]},
  {'id': [33183, 351730],
   'text': ['Which is the best smartphone under 20K in India?',
    'Which is the best smartphone with in 20k in India?']},
  {'id': [351731, 351732],
   'text': ['Steps taken by Canadian government to improve literacy rate?',
    'Can I send homemade herbal hair oil from India to US via postal or private courier services?']},
  {'id': [37799, 94186],
   'text': ['What is a good way to lose 30 pounds in 2 months?',
    'What can I do to lose 30 pounds in 2 months?']},
  {'id': [351733, 351734],
   'text': ['Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?',
    'How do you graph x + 2y = -2?']}],
 'is_duplicate': [False, True, False, True, False]}

In [6]:
questions = []
for record in dataset['questions']:
    questions.extend(record['text'])
    
question = list(set(questions))
print('\n'.join(questions[:10]))
print('-' * 50)
print(f'Number of questions: {len(questions)}')

What is the truth of life?
What's the evil truth of life?
Which is the best smartphone under 20K in India?
Which is the best smartphone with in 20k in India?
Steps taken by Canadian government to improve literacy rate?
Can I send homemade herbal hair oil from India to US via postal or private courier services?
What is a good way to lose 30 pounds in 2 months?
What can I do to lose 30 pounds in 2 months?
Which of the following most accurately describes the translation of the graph y = (x+3)^2 -2 to the graph of y = (x -2)^2 +2?
How do you graph x + 2y = -2?
--------------------------------------------------
Number of questions: 100000


### Check cuda and Setup the model

**Note**: "Checking cuda" refers to checking if you have access to GPUs (faster compute). In this course, we are using CPUs. So, you might notice some code cells taking a little longer to run.

We are using *all-MiniLM-L6-v2* sentence-transformers model that maps sentences to a 384 dimensional dense vector space.

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
    
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

Sorry no cuda.


In [8]:
query = 'which city is the most populated in the entire world?'
xq = model.encode(query)
xq.shape

(384,)

### Setup Pinecone

In [10]:
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()
# print(PINECONE_API_KEY)

In [12]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = utils.create_dlai_index_name('dl-ai')

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)
print(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, 
    dimension=model.get_sentence_embedding_dimension(), 
    metric='cosine',
    spec=ServerlessSpec(cloud='aws', region='us-west-2'))

index = pinecone.Index(INDEX_NAME)
print(index)

dl-ai-qy4e9es1t3blbkfj2bmmjhs4ton6dw4ghvds


### Create Embeddings and Upsert to Pinecone

In [13]:
batch_size=200
vector_limit=10000

questions = question[:vector_limit]

import json

for i in tqdm(range(0, len(questions), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(questions))
    
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    
    # create metadata batch
    metadatas = [{'text': text} for text in questions[i:i_end]]
    
    # create embeddings
    xc = model.encode(questions[i:i_end])
    
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    
    # upsert to Pinecone
    index.upsert(vectors=records)

  0%|          | 0/50 [00:00<?, ?it/s]

In [14]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10000}},
 'total_vector_count': 10000}

### Run Your Query

In [15]:
# small helper function so we can repeat queries later
def run_query(query):
  embedding = model.encode(query).tolist()
  results = index.query(top_k=10, vector=embedding, include_metadata=True, include_values=False)
  for result in results['matches']:
    print(f"{round(result['score'], 2)}: {result['metadata']['text']}")

In [16]:
run_query('which city has the highest altitude in the world?')

0.76: What's the highest mountain in the world?
0.67: What is the highest mountain in India?
0.47: Which is the largest country in the world?
0.46: Why does temperature decrease when altitude increases?
0.43: At high altitudes, the RBCs in the human blood increase in number. Why?
0.42: What is the most beautiful place to visit in Singapore?
0.42: What is K.E of a projectile at its maximum height?
0.42: Which are the most dangerous places on the earth to live? Why?
0.42: Which is the lowest layer of the atmosphere?
0.42: What do you think is the greatest country in the world?


In [17]:
query = 'how do i make banana cake?'
run_query(query)

0.61: How do you add chocolate chips to cake mix?
0.59: What are some ways to add chocolate chips to a cake mix?
0.55: How do I make rice?
0.54: How do you make baking soda?
0.49: Does a banana increase fat?
0.47: How is a tomato a fruit and a banana a vegetable?
0.47: What are some dessert recipes to prepare with sour green apples?
0.47: How do you make a perfume out of Skittles?
0.43: Is it fine to eat four bananas in a day?
0.43: Where can I buy best quality customized cupcakes in Gold Coast?
